In [103]:
import numpy as np
from sklearn.cluster import DBSCAN, KMeans
import re
import nibabel as nib
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd
import os, shutil
import pickle as cPickle
import ipycache

In [104]:
def printLabels(folderName, labels,originalPaths):
    if os.path.exists(folderName):
        shutil.rmtree(folderName)
    os.mkdir(folderName)
    fileNamesDict = {}
    print(labels)
    for lineNum, label in enumerate(labels):
        fileName = "".join((folderName,"/",str(label),".txt"))
        if fileName in fileNamesDict:
            file = fileNamesDict[fileName]
        else:
            file =  open(fileName, "w+")
            fileNamesDict[fileName] = file
        originalPaths[lineNum].astype(np.int)
        line = str(originalPaths[lineNum])[1:-1] #flattened paths contain the unedited paths hashed by the root node
        line = re.sub("'|\n|[|]", "", line) #get rid of junk characters when converting from an array to a string
        file.write(" ".join((line,"\n")))
    allLabelFiles = open("".join(("all",folderName,"Files.txt")),"w+")
    allLabelFiles.write("bv_loadDataImage      SWI_Images_resize10.nii \n")
    allLabelFiles.write("bv_loadSegmentedImage csf_seg_8bit_resize10_8bit.nii \n")
    for fileKeys in fileNamesDict:
        allLabelFiles.write("    ".join(("bv_readPathsFromFile",fileNamesDict[fileKeys].name)))
        allLabelFiles.write("\n")
        fileNamesDict[fileKeys].close()
    #allPathFiles.write(str(clusteringMethod))
    allLabelFiles.close()

In [105]:
%load_ext ipycache

The ipycache extension is already loaded. To reload it, use:
  %reload_ext ipycache


In [106]:
DEBUG = 0 #toggle for print statements
EXPLORE = 0 #toggle for ploting clusters and seeing number of components with pca
USING_ROOT_BUNCHING = 0 #toggle between modes of operation
ZERO_PADDED = 0 #set to 1 to pad the intensities with 0s 

In [158]:
paths = len(open("pathFile1.txt","r").readlines()) if DEBUG else None
longestPathLength = int((len(max(open("pathFile1.txt","r"),key=len).split(" "))-1)/3) if DEBUG else None
print(paths) if DEBUG == 1 else None
print(longestPathLength) if DEBUG else None
dilationRadius = 0 #has to be greater than or equal to 0
numIntensityParams = ((2*(dilationRadius+1)-1)**2) #number of intensities to use
if dilationRadius <0 :
    numIntensityParams = 0
print(numIntensityParams) if DEBUG else None

In [165]:
%%cache imageData.pkl swiImage swiData
swiImage = nib.load("SWI_Images_resize10.nii")
swiImage.shape
swiData = swiImage.get_fdata()
print(swiData.shape) if DEBUG else None


#swiData = swiData/np.max(np.abs(swiData),axis=0) #normalize by the max across entire image

[Skipped the cell's code and loaded variables swiData, swiImage from file 'C:\Users\Linghai Wang\Documents\GPN\Clustering\imageData.pkl'.]


In [173]:
print(np.max(swiData))

622.0


In [160]:
%%cache loadedAllPaths allPaths allOriginalPaths allIntensityPaths
if not USING_ROOT_BUNCHING:
    pathFile = open("pathFile1.txt","r")
    allPaths = []
    allOriginalPaths = []
    allIntensityPaths = []
    for lineNum, line in tqdm(enumerate(pathFile)):
        currentSplitLine = line.split(" ")
        currentSplitLine = currentSplitLine[:-1]
        flatLine = currentSplitLine
        allOriginalPaths.append(flatLine)

        currentSplitLine = list(map(int,currentSplitLine))

        gFoldedLine = np.reshape(currentSplitLine,(int(len(currentSplitLine)/3),3))

        paddedFoldedLine = np.pad(gFoldedLine,[(0,0),(0,numIntensityParams)], "constant", constant_values = 0)
        paddedFoldedLine = np.delete(paddedFoldedLine, 0, 1)#the x value is useless since its just the index of the array so its just noise
        intensityLine = np.zeros((paddedFoldedLine.shape[0],numIntensityParams))
        if dilationRadius>=0:
            for pixelNum, pixel in enumerate(gFoldedLine):
                for i in range(numIntensityParams):
                    x = int(pixel[0])
                    y = int(pixel[1])-dilationRadius+int((i%(2*dilationRadius-1)))
                    z = int(pixel[2])-dilationRadius+int(i/(2*dilationRadius-1))

                    if x<0 or y<0 or z<0 or x>=swiData.shape[0] or y >= swiData.shape[1] or z >= swiData.shape[2]:
                        if ZERO_PADDED:
                            pixelIntensity = 0 #622 is max intensity #essentially pad with 0s if the shell goes out of bounds
                        else:
                            if x<0:
                                x = 0
                            if y<0:
                                y = 0
                            if z<0:
                                z = 0
                            if x>=swiData.shape[0]:
                                x = swiData.shape[0]
                            if y>=swiData.shape[1]:
                                y = swiData.shape[1]
                            if z>=swiData.shape[2]:
                                z = swiData.shape[2]
                    else:
                        pixelIntensity = swiData[x][y][z]
                    paddedFoldedLine[pixelNum][2+i] = pixelIntensity
                    intensityLine[pixelNum][i] = pixelIntensity                    
        allPaths.append(paddedFoldedLine)
        allIntensityPaths.append(intensityLine)
    pathFile.close()

[Saved variables 'allIntensityPaths, allOriginalPaths, allPaths' to file 'C:\Users\Linghai Wang\Documents\GPN\Clustering\loadedAllPaths'.]


1575it [00:00, 2185.01it/s]


In [110]:
%%cache loadedBunchPaths.pkl bunchedPaths flatennedPaths bunchedIntensities
if USING_ROOT_BUNCHING:
    pathFile = open("pathFile1.txt","r")
    # start with just the xyz and intensity of the paths nothing around it yet
    bunchedPaths = {}#dictionary with the keys as the root node as a joined spring and the values as a list of all paths with that root node
    flatennedPaths = {}#used for initial clustering without intesities 
    bunchedIntensities = {} # no x,y,z just intensities
    for lineNum, line in tqdm(enumerate(pathFile)):
        currentSplitLine = line.split(" ")
        currentSplitLine = currentSplitLine[:-1]
        flatLine = currentSplitLine
        currentSplitLine = list(map(float,currentSplitLine))
        gFoldedLine = np.reshape(currentSplitLine,(int(len(currentSplitLine)/3),3))


        rootNode = " ".join(list(map(str,map(int,gFoldedLine[58]))))
        gFoldedLine.astype(np.float)
        groupArray = []
        flatGroupArray = []
        intensityArray = []
        if rootNode in bunchedPaths: # if the root hasnt been added to the dictionary add it to the dictionary with a list of paths
            groupArray = bunchedPaths[rootNode] #join the string to hash a string instead of a tuple since python likes that better
            flatGroupArray = flatennedPaths[rootNode]
            intensityArray = bunchedIntensities[rootNode]
        else:
            bunchedPaths[rootNode] = groupArray
            flatennedPaths[rootNode] =flatGroupArray 
            bunchedIntensities[rootNode] = intensityArray

        flatGroupArray.append(flatLine)


        #gFoldedLine =gFoldedLine/np.max(gFoldedLine,axis=0) #currently normalizaing along the path to capture the variance within a single path but not across each root or the entire image
        paddedFoldedLine = np.pad(gFoldedLine,[(0,0),(0,numIntensityParams)], "constant", constant_values = 0)
        paddedFoldedLine = np.delete(paddedFoldedLine, 0, 1)#the x value is useless since its just the index of the array so its just noise
        intensityLine = np.zeros((paddedFoldedLine.shape[0],numIntensityParams))
        if dilationRadius>=0:
            for pixelNum, pixel in enumerate(gFoldedLine):
                for i in range(numIntensityParams):
                    x = int(pixel[0])
                    y = int(pixel[1])-dilationRadius+int((i%(2*dilationRadius-1)))
                    z = int(pixel[2])-dilationRadius+int(i/(2*dilationRadius-1))
                    
                    if x<0 or y<0 or z<0 or x>=swiData.shape[0] or y >= swiData.shape[1] or z >= swiData.shape[2]:
                        
                        pixelIntensity = 0 #essentially pad with 0s if the shell goes out of bounds
                    else:
                        pixelIntensity = swiData[x][y][z]
                    paddedFoldedLine[pixelNum][2+i] = pixelIntensity
                    intensityLine[pixelNum][i] = pixelIntensity

        paddedFoldedLine.astype(np.float)
        groupArray.append(paddedFoldedLine)
        intensityArray.append(intensityLine)
        print(paddedFoldedLine.shape) if DEBUG else None
    pathFile.close()

[Saved variables 'bunchedIntensities, bunchedPaths, flatennedPaths' to file 'C:\Users\Linghai Wang\Documents\GPN\Clustering\loadedBunchPaths.pkl'.]


In [111]:
print(bunchedIntensities.keys()) if DEBUG else None
#21 unique roots

In [112]:
print(flatennedPaths.keys()) if DEBUG else None

In [113]:
if USING_ROOT_BUNCHING:
    #just to make sure
    for keys in bunchedPaths:
        bunchedPaths[keys] = np.array(bunchedPaths[keys])#.astype(np.float)
        print(bunchedPaths[keys].shape)if DEBUG else None

In [114]:
if USING_ROOT_BUNCHING:
    #just to make sure
    for keys in bunchedIntensities:
        bunchedIntensities[keys] = np.array(bunchedIntensities[keys])#.astype(np.float)
        print(bunchedIntensities[keys].shape) if DEBUG else None

In [115]:
if USING_ROOT_BUNCHING:
    for keys in flatennedPaths:
        flatennedPaths[keys] = np.array(flatennedPaths[keys])#.astype(np.float)
        print(flatennedPaths[keys]) if DEBUG else None

In [116]:
if USING_ROOT_BUNCHING:
    clusteringMethod = DBSCAN(eps = 3, min_samples = 2) #DBSCAN(eps=0.5, min_samples=5, metric=’euclidean’, metric_params=None, algorithm=’auto’, leaf_size=30, p=None, n_jobs=None)[source]¶
    labels = {}
    for keys in flatennedPaths:
        clusteringMethod.fit(flatennedPaths[keys])
        labels[keys] = clusteringMethod.labels_
        #print(flatennedPaths[keys].shape)

In [117]:
print(labels) if DEBUG else None

In [118]:
if EXPLORE:
    #this cell gets the variance ratios for all possible numbers of components for each root node
    varianceDictionary = {}
    for keys in bunchedPaths:
        currentBunch = bunchedPaths[keys]
        currentBunch = np.reshape(currentBunch, (currentBunch.shape[0], currentBunch.shape[1]*currentBunch.shape[2])) #flatten
        currentBunch = normalize(currentBunch,norm = 'l2', axis = 0) #normalize by root node by a squared average
        varianceList = []
        for i in range(min(currentBunch.shape[0],currentBunch.shape[1])):
            pca = PCA(n_components = i)
            pca.fit(currentBunch)
            varianceList.append(np.sum(pca.explained_variance_ratio_))
        

In [119]:
if USING_ROOT_BUNCHING:    
    intensityLabels = {} #intensity+x,y,z
    for keys in tqdm(bunchedPaths):
        clusteringMethod = DBSCAN(eps = .1, min_samples = 2)
        currentBunch = bunchedPaths[keys]
        currentBunch = np.reshape(currentBunch, (currentBunch.shape[0], currentBunch.shape[1]*currentBunch.shape[2])) #flatten
        currentBunch = normalize(currentBunch,norm = 'l2', axis = 0) #normalize by root node by a squared average
        pca = PCA(.99)
        pca.fit(currentBunch)
        transformed = pca.transform(currentBunch)
        print(currentBunch.shape) if DEBUG else None
        clusteringMethod.fit(transformed)
        intensityLabels[keys] = clusteringMethod.labels_

In [120]:
if USING_ROOT_BUNCHING:
    if dilationRadius > 0:
        intensityClusters = {} #just intensity
        for keys in tqdm(bunchedIntensities):
            clusteringMethod = DBSCAN(eps = .1, min_samples = 2)
            currentBunch = bunchedIntensities[keys]
            if numIntensityParams >1:
                currentBunch = np.reshape(currentBunch, (currentBunch.shape[0], currentBunch.shape[1]*currentBunch.shape[2])) #flatten
            currentBunch = normalize(currentBunch,norm = 'l2', axis = 0) #normalize by root node by a squared average
            pca = PCA(.99)
            pca.fit(currentBunch)
            transformed = pca.transform(currentBunch)
            print(currentBunch.shape) if DEBUG else None
            clusteringMethod.fit(transformed)
            intensityClusters[keys] = clusteringMethod.labels_

In [121]:
#this cell varys the eps to see the change in the number of clusters with pca set to .95 variance ratio
if EXPLORE:
    clusterDict = {}
    for keys in bunchedPaths:
        clusterList = []
        for i in tqdm(range(10)):
            clusteringMethod = DBSCAN(eps = (i+1)/10, min_samples = 2) #DBSCAN(eps=0.5, min_samples=5, metric=’euclidean’, metric_params=None, algorithm=’auto’, leaf_size=30, p=None, n_jobs=None)[source]¶
            currentBunch = bunchedPaths[keys]
            currentBunch = np.reshape(currentBunch, (currentBunch.shape[0], currentBunch.shape[1]*currentBunch.shape[2])) #flatten
            currentBunch = normalize(currentBunch,norm = 'l2', axis = 0) #normalize by root node by a squared average
            for j in range(1,min(currentBunch.shape[0],currentBunch.shape[1])):
                pca = PCA(n_components = j)
                pca.fit(currentBunch)
                transformed = pca.transform(currentBunch)
                print(currentBunch.shape) if DEBUG else None
                clusteringMethod.fit(transformed)
                unique, counts = np.unique(clusteringMethod.labels_, return_counts=True)
                mapping = dict(zip(unique, counts))
                outliers = 0;
                if -1 in mapping:
                    outliers = mapping[-1]
                uniqueClusters = len(unique) + outliers
                clusterList.append(((i+1)/10,j,np.sum(pca.explained_variance_ratio_),uniqueClusters,outliers))

        clusterDict[keys] = clusterList

In [122]:
print(intensityLabels) if DEBUG else None

In [123]:
%matplotlib inline
if EXPLORE:
    print(clusterDict.keys())
    data = clusterDict['21 5 3']
    df = pd.DataFrame(data, columns = ['eps', 'numcomponents', 'variance ratio','num clusters','outliers'])
    print(df) if DEBUG else None
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(df['eps'], df['numcomponents'], df['num clusters'], c='skyblue', s=15)
    plt.show()


In [124]:
if USING_ROOT_BUNCHING:
    if os.path.exists('pathClusters'):
        shutil.rmtree('pathClusters')
    os.mkdir("pathClusters")
    pathFiles = {}
    for currentKey in intensityLabels:#currently uses the intensityLabels instead of the standard labels from clustering
        for index, cluster in enumerate(intensityLabels[currentKey]):
            clusterRoot = currentKey.split(" ")
            clusterRoot = ",".join(clusterRoot)
            filename = "".join(("pathClusters/",clusterRoot,";",str(cluster),".txt"))#format the filename based on the root node
            if filename in pathFiles:
                workingFile = pathFiles[filename]
            else:
                workingFile =  open(filename, "w+")
                pathFiles[filename] = workingFile
            print(str(flatennedPaths[currentKey][cluster])) if DEBUG else None
            flatennedPaths[currentKey][index].astype(np.int)
            line = str(flatennedPaths[currentKey][index])[1:-1] #flattened paths contain the unedited paths hashed by the root node
            line = re.sub("'|\n|[|]", "", line) #get rid of junk characters when converting from an array to a string
            workingFile.write(" ".join((line,"\n")))
else:
    pass

In [125]:
if USING_ROOT_BUNCHING:    
    allBunchedPathFiles = open("BunchedPathFiles.txt","w+")
    allBunchedPathFiles.write("bv_loadDataImage      SWI_Images_resize10.nii \n")
    allBunchedPathFiles.write("bv_loadSegmentedImage csf_seg_8bit_resize10_8bit.nii \n")
    for fileKeys in pathFiles:
        allBunchedPathFiles.write("    ".join(("bv_readPathsFromFile",pathFiles[fileKeys].name)))
        allBunchedPathFiles.write("\n")
        pathFiles[fileKeys].close()
    #allPathFiles.write(str(clusteringMethod))
    allBunchedPathFiles.close()

In [126]:
if USING_ROOT_BUNCHING:
    if os.path.exists('intesnsityClusters'):
        shutil.rmtree('intesnsityClusters')
    os.mkdir("intesnsityClusters")
    intensityFiles = {}
    for currentKey in intensityLabels:#currently uses the intensityLabels instead of the standard labels from clustering
        for index, cluster in enumerate(intensityLabels[currentKey]):
            clusterRoot = currentKey.split(" ")
            clusterRoot = ",".join(clusterRoot)
            filename = "".join(("intesnsityClusters/",clusterRoot,";",str(cluster),".txt"))#format the filename based on the root node
            if filename in intensityFiles:
                workingFile = intensityFiles[filename]
            else:
                workingFile =  open(filename, "w+")
                intensityFiles[filename] = workingFile
            #print(str(flatennedPaths[currentKey][cluster]))
            flatennedPaths[currentKey][index].astype(np.int)
            line = str(flatennedPaths[currentKey][index])[1:-1] #flattened paths contain the unedited paths hashed by the root node
            line = re.sub("'|\n|[|]", "", line) #get rid of junk characters when converting from an array to a string
            workingFile.write(" ".join((line,"\n")))


In [127]:
if USING_ROOT_BUNCHING:
    allBunchedIntensityFiles = open("BunchedIntensityFiles.txt","w+")
    allBunchedIntensityFiles.write("bv_loadDataImage      SWI_Images_resize10.nii \n")
    allBunchedIntensityFiles.write("bv_loadSegmentedImage csf_seg_8bit_resize10_8bit.nii \n")
    for fileKeys in intensityFiles:
        allBunchedIntensityFiles.write("    ".join(("bv_readPathsFromFile",intensityFiles[fileKeys].name)))
        allBunchedIntensityFiles.write("\n")
        intensityFiles[fileKeys].close()
    #allPathFiles.write(str(clusteringMethod))
    allBunchedIntensityFiles.close()

In [128]:
if USING_ROOT_BUNCHING:
    bunchFile = open("bunchedPaths.txt","w+")
    for keys in bunchedPaths:
        for line in bunchedPaths[keys]:
            for pixelSet in line:
                bunchFile.write(re.sub("'|\n|[|]", "", str(pixelSet)))
                bunchFile.write("\n")
            bunchFile.write("\n")
    bunchFile.close

In [161]:
if not USING_ROOT_BUNCHING:
    allPaths = np.array(allPaths)
    allOriginalPaths = np.array(allOriginalPaths)
    allIntensityPaths = np.array(allIntensityPaths)
    print(allIntensityPaths.shape)

(1575, 59, 1)


In [130]:
if not USING_ROOT_BUNCHING:
    clusteringMethod = DBSCAN(eps = .1, min_samples = 2)
    flatAllPaths = np.reshape(allPaths, (allPaths.shape[0], allPaths.shape[1]*allPaths.shape[2])) #flatten
    flatAllPaths = normalize(flatAllPaths,norm = 'l2', axis = 0)   
    pca = PCA(.95)
    pca.fit(flatAllPaths)
    transformed = pca.transform(flatAllPaths)
    clusteringMethod.fit(transformed)
    allPathsLabels = clusteringMethod.labels_

In [131]:
if not USING_ROOT_BUNCHING:
    clusteringMethod = DBSCAN(eps = .1, min_samples = 2)
    flatAllOriginalPaths = normalize(allOriginalPaths,norm = 'l2', axis = 0)   
    pca = PCA(.95)
    pca.fit(flatAllOriginalPaths)
    transformed = pca.transform(flatAllOriginalPaths)
    clusteringMethod.fit(transformed)
    allOriginalPathsLabels = clusteringMethod.labels_

In [132]:
if not USING_ROOT_BUNCHING:
    clusteringMethod = DBSCAN(eps = .1, min_samples = 2)
    flatAllIntensityPaths = np.reshape(allIntensityPaths, (allIntensityPaths.shape[0], allIntensityPaths.shape[1]*allIntensityPaths.shape[2])) #flatten
    flatAllIntensityPaths = normalize(flatAllIntensityPaths,norm = 'l2', axis = 0)   
    pca = PCA(.95)
    pca.fit(flatAllIntensityPaths)
    transformed = pca.transform(flatAllIntensityPaths)
    clusteringMethod.fit(transformed)
    allIntensityPathsLabels = clusteringMethod.labels_

In [133]:
if not USING_ROOT_BUNCHING:
    if os.path.exists('allPathClusters'):
        shutil.rmtree('allPathClusters')
    os.mkdir("allPathClusters")
    allPathsDict = {}
    for lineNum, label in enumerate(allPathsLabels):
        fileName = "".join(("allPathClusters/",str(label),".txt"))
        if fileName in allPathsDict:
            file = allPathsDict[fileName]
        else:
            file =  open(fileName, "w+")
            allPathsDict[fileName] = file
        allOriginalPaths[lineNum].astype(np.int)
        line = str(allOriginalPaths[lineNum])[1:-1] #flattened paths contain the unedited paths hashed by the root node
        line = re.sub("'|\n|[|]", "", line) #get rid of junk characters when converting from an array to a string
        file.write(" ".join((line,"\n")))

In [134]:
if not USING_ROOT_BUNCHING:
    allPathFiles = open("allPathFiles.txt","w+")
    allPathFiles.write("bv_loadDataImage      SWI_Images_resize10.nii \n")
    allPathFiles.write("bv_loadSegmentedImage csf_seg_8bit_resize10_8bit.nii \n")
    for fileKeys in allPathsDict:
        allPathFiles.write("    ".join(("bv_readPathsFromFile",allPathsDict[fileKeys].name)))
        allPathFiles.write("\n")
        allPathsDict[fileKeys].close()
    #allPathFiles.write(str(clusteringMethod))
    allPathFiles.close()

In [135]:
if not USING_ROOT_BUNCHING:
    if os.path.exists('allOriginalPathCLusters'):
        shutil.rmtree('allOriginalPathCLusters')
    os.mkdir("allOriginalPathCLusters")
    allOriginalPathsDict = {}
    for lineNum, label in enumerate(allOriginalPathsLabels):
        fileName = "".join(("allOriginalPathCLusters/",str(label),".txt"))
        if fileName in allOriginalPathsDict:
            file = allOriginalPathsDict[fileName]
        else:
            file =  open(fileName, "w+")
            allOriginalPathsDict[fileName] = file
        allOriginalPaths[lineNum].astype(np.int)
        line = str(allOriginalPaths[lineNum])[1:-1] #flattened paths contain the unedited paths hashed by the root node
        line = re.sub("'|\n|[|]", "", line) #get rid of junk characters when converting from an array to a string
        file.write(" ".join((line,"\n")))

In [136]:
if not USING_ROOT_BUNCHING:
    allOriginalPathFiles = open("allOriginalPathFiles.txt","w+")
    allOriginalPathFiles.write("bv_loadDataImage      SWI_Images_resize10.nii \n")
    allOriginalPathFiles.write("bv_loadSegmentedImage csf_seg_8bit_resize10_8bit.nii \n")
    for fileKeys in allOriginalPathsDict:
        allOriginalPathFiles.write("    ".join(("bv_readPathsFromFile",allOriginalPathsDict[fileKeys].name)))
        allOriginalPathFiles.write("\n")
        allOriginalPathsDict[fileKeys].close()
    #allPathFiles.write(str(clusteringMethod))
    allOriginalPathFiles.close()

In [137]:
if not USING_ROOT_BUNCHING:
    if os.path.exists('allIntensityPathsClusters'):
        shutil.rmtree('allIntensityPathsClusters')
    os.mkdir("allIntensityPathsClusters")
    allIntensitylPathsDict = {}
    print(allIntensityPathsLabels)
    for lineNum, label in enumerate(allIntensityPathsLabels):
        fileName = "".join(("allIntensityPathsClusters/",str(label),".txt"))
        if fileName in allIntensitylPathsDict:
            file = allIntensitylPathsDict[fileName]
        else:
            file =  open(fileName, "w+")
            allIntensitylPathsDict[fileName] = file
        allOriginalPaths[lineNum].astype(np.int)
        line = str(allOriginalPaths[lineNum])[1:-1] #flattened paths contain the unedited paths hashed by the root node
        line = re.sub("'|\n|[|]", "", line) #get rid of junk characters when converting from an array to a string
        file.write(" ".join((line,"\n")))

[-1  0  0 ...  0  0  0]


In [138]:
if not USING_ROOT_BUNCHING:
    allIntensityPathFiles = open("allIntensityPathFiles.txt","w+")
    allIntensityPathFiles.write("bv_loadDataImage      SWI_Images_resize10.nii \n")
    allIntensityPathFiles.write("bv_loadSegmentedImage csf_seg_8bit_resize10_8bit.nii \n")
    for fileKeys in allIntensitylPathsDict:
        allIntensityPathFiles.write("    ".join(("bv_readPathsFromFile",allIntensitylPathsDict[fileKeys].name)))
        allIntensityPathFiles.write("\n")
        allIntensitylPathsDict[fileKeys].close()
    #allPathFiles.write(str(clusteringMethod))
    allIntensityPathFiles.close()

In [162]:
if not USING_ROOT_BUNCHING:
    clusteringMethod = KMeans(n_clusters = 6)
    xSlice = np.reshape(allIntensityPaths[:,0], (allIntensityPaths.shape[0], numIntensityParams)) #flatten
    #xSlice = normalize(xSlice,norm = 'l1', axis = 0)   
    clusteringMethod.fit(xSlice)
    xSliceLabels = clusteringMethod.labels_
    print(clusteringMethod.cluster_centers_)
    print(xSliceLabels)

[[278.23419204]
 [360.61271676]
 [316.81422925]
 [ 83.10344828]
 [424.40677966]
 [229.16778523]]
[0 5 4 ... 1 1 1]


In [163]:
print(allOriginalPaths[:,0:3])
printLabels("justXSlice",xSliceLabels,allOriginalPaths[:,0:3])

[['79' '0' '0']
 ['79' '1' '0']
 ['79' '2' '0']
 ...
 ['79' '102' '14']
 ['79' '103' '14']
 ['79' '104' '14']]
[0 5 4 ... 1 1 1]


In [141]:
printLabels("testFolder",allIntensityPathsLabels,allOriginalPaths)

[-1  0  0 ...  0  0  0]


In [142]:
a = [1,2,3,4,5,6,7,8,9,10]
a =np.reshape(a,(5,2))
print(a) if DEBUG else None
#a = np.pad(a,[(0,0),(0,5)], "constant", constant_values = 0 )
#print(a)
a = a/np.max(a,axis=0)
print(a) if DEBUG else None

In [143]:
print(swiData[68][8][1]) if DEBUG else None

In [144]:
for i in range(numIntensityParams):
    y = int((i%(2*dilationRadius-1)))
    z = int(i/(2*dilationRadius-1))
    print(y,z, sep = " ")
print(numIntensityParams) if DEBUG else None

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8


In [145]:
a = [[4, 1, 2, 2],
...  [1, 3, 9, 3],
...  [5, 7, 5, 1]]
b = normalize(a, norm='l1', axis=0)#l1 is a average
print(b) if DEBUG else None
pca = PCA(.9)
pca.fit(b)
c =pca.transform(b)
print(c) if DEBUG else None

In [146]:
%%time
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print(arr) if DEBUG else None
arr = np.delete(arr, 0, 1)
print(arr) if DEBUG else None

Wall time: 0 ns
